1 - Importação dos pacotes e bibliotecas

In [ ]:
# Usada para manipulação de dados
import pandas as pd

# Usado para tranformar strings em dados numéricos
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Módulos do Scikit-learn usado para algoritimos de regressão
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Módulo do Scikit-learn usado para separar o conjunto de dados.
from sklearn.model_selection import train_test_split

# Módulos do Scikit-learn usado para avaliação estatística.
from sklearn.metrics  import mean_absolute_error
from sklearn.metrics  import mean_squared_error
from sklearn.metrics  import r2_score

2 - Ingenstão dos dados de entrega

In [ ]:
# Define o Caminho dos arquivos.
csv_customers = "../data/olist_customers_dataset.csv"
csv_order_itens = "../data/olist_order_items_dataset.csv"
csv_orders = "../data/olist_orders_dataset.csv"
csv_products = "../data/olist_products_dataset.csv"
csv_sellers = "../data/olist_sellers_dataset.csv"

# Carrega os arquivos CSV como DataFrames
df_costumers = pd.read_csv(csv_customers)
df_order_itens = pd.read_csv(csv_order_itens)
df_orders = pd.read_csv(csv_orders)
df_products = pd.read_csv(csv_products)
df_sellers = pd.read_csv(csv_sellers)

# Junta os dataframes em um unico dataframe para utilizar no machine learning.
df = df_orders.merge(df_order_itens, on="order_id")
df = df.merge(df_costumers, on="customer_id")
df = df.merge(df_sellers, on='seller_id')
df = df.merge(df_products, on="product_id")


# Exibe o dataframe
df.head()

3 - Tratamento dos dados

In [ ]:
# Tira todas as colunas irrelevantes para o aprendizado de maquina.
df = df.drop(['order_id', 'customer_id','order_status', 'product_id', 'order_item_id', 'seller_id', 'customer_unique_id', 'product_name_lenght', 'product_description_lenght', "product_photos_qty", "order_approved_at", 'order_delivered_carrier_date', 'order_estimated_delivery_date', 'shipping_limit_date', 'customer_zip_code_prefix', 'seller_zip_code_prefix'], axis=1)


In [ ]:
# Remove as linhas que contenham algum valor nulo.
df = df.dropna()

# Trasforma as colunas com strings em valores numeros para o aprendizado de maquina.
colunas_categoricas = ['customer_state', 'seller_state', 'customer_city', 'seller_city', 'product_category_name']
for col in colunas_categoricas:
    df[col] = le.fit_transform(df[col].astype(str))


In [ ]:
# Converte as datas para datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Cria a Variável alvo: diferença em dias entre a entrega e a compra.
df['prazo_entrega_dias'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp'])

# Converte para um dado numérico.
df['prazo_entrega_dias'] = df['prazo_entrega_dias'].dt.days

# Remove os outliers de tempo de entrega.
df = df[df['prazo_entrega_dias'] <= 30]

# Cria colunas com o dia da semana de compra, o mês e a hora.
df['order_weekday'] = df['order_purchase_timestamp'].dt.weekday # 0 = segunda.
df['order_month'] = df['order_purchase_timestamp'].dt.month
df['order_hour'] = df['order_purchase_timestamp'].dt.hour

# Exclui as colunas de data de compra e data de entrega.
df = df.drop(['order_purchase_timestamp', 'order_delivered_customer_date'] , axis=1)

In [ ]:
# Exibe o dataframe
display(df)

# Exibe informações detalhadas sobre a estrutura dos dados.
display(df.info())

# Exibe a quantidade de valores nulos por coluna.
print(df.isnull().sum())


4 - Separação dos dados     

In [ ]:
# Divide os dados para treinamento e teste.
df_treino, df_teste = train_test_split(df, test_size=20, random_state=42)

# Define o alvo e os preditores de treinamento.
y_treino = df_treino['prazo_entrega_dias']
x_treino = df_treino[['price', 'freight_value', 'customer_city', 'customer_state', 'seller_city', 'seller_state', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'order_weekday', 'order_month', 'order_hour']]

# Define o alvo e os preditores de teste.
y_teste = df_teste['prazo_entrega_dias']
x_teste = df_teste[['price', 'freight_value', 'customer_city', 'customer_state', 'seller_city', 'seller_state', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'order_weekday', 'order_month', 'order_hour']]

5 - Treinamento dos modelos de machine learning

In [ ]:
# Cria o modelo de regressão linear
RL = LinearRegression()

# Treina o modelo de reguressao linear
RL.fit(x_treino, y_treino)

# Cria o modelo de Gradient Bossting
GB = GradientBoostingRegressor()

# Treina o modelo de Gradient Bossting
GB.fit(x_treino, y_treino)

# Cria o modelo de Random Forest
RF = RandomForestRegressor()

# Treina o modelo de Rando Forest
RF.fit(x_treino, y_treino)

6 - Avaliação Estatística dos Resultados.

In [ ]:
# Calcula as Métricas estatísticas.
RL_mbe = mean_absolute_error(y_teste, RL.predict(x_teste))

RL_rmbe = mean_squared_error(y_teste, RL.predict(x_teste))

RL_r2 = r2_score(y_teste, RL.predict(x_teste))

# Exibe os Resultados.
print('Resultados do modelo de Regressão Linear:')
# Erro Médio (MBE).
print(f'MBE: {RL_mbe:.2f}')
# Raiz do erro Quadrático medio (RMSE).
print(f'RMSE: {RL_rmbe:.2f}')
# Coeficiente de Determinação (R2).
print(f'R²: {RL_r2:.2f}')

# Calcula as Métricas estatísticas.
GB_mbe = mean_absolute_error(y_teste, GB.predict(x_teste))

GB_rmbe = mean_squared_error(y_teste, GB.predict(x_teste))

GB_r2 = r2_score(y_teste, GB.predict(x_teste))

# Exibe os Resultados.
print('Resultados do modelo de Gradient Boosting:')
# Erro Médio (MBE).
print(f'MBE: {GB_mbe:.2f}')
# Raiz do erro Quadrático medio (RMSE).
print(f'RMSE: {GB_rmbe:.2f}')
# Coeficiente de Determinação (R2).
print(f'R²: {GB_r2:.2f}')


# Calcula as Métricas estatísticas.
RF_mbe = mean_absolute_error(y_teste, RF.predict(x_teste))

RF_rmbe = mean_squared_error(y_teste, RF.predict(x_teste))

RF_r2 = r2_score(y_teste, RF.predict(x_teste))

# Exibe os Resultados.
print('Resultados do modelo de Random Forest:')
# Erro Médio (MBE).
print(f'MBE: {RF_mbe:.2f}')
# Raiz do erro Quadrático medio (RMSE).
print(f'RMSE: {RF_rmbe:.2f}')
# Coeficiente de Determinação (R2).
print(f'R²: {RF_r2:.2f}')

